**Install OTXv2 package**

In [ ]:
pip install OTXv2

**IOC collection from OTXv2**

In [ ]:
#IOC collection
from OTXv2 import OTXv2, IndicatorTypes
from pandas.io.json import json_normalize
from datetime import datetime, timedelta
import json
otx = OTXv2("3e1d17b591d8e946c88e4dd05919eecba708d60c674ab796490d184bfd96e0a1")

# Get the latest pulse in the 'Manufacturing' industry category
# manufacturing_pulses = otx.search_pulses("manufacturing")
# pulse_details = otx.get_pulse_details('63d78854ff0f2e48acb6ab50')

pulses = otx.getall()


with open("ICS_IOCs_updated.json", "w") as f:
    json.dump(pulses, f, indent=4)

ModuleNotFoundError: No module named 'OTXv2'

**Embedding generation (Feature matrix)**

In [1]:
from scipy.special import chdtriv
import json
from gensim.models import Word2Vec
import numpy as np


with open('ICS_IOCs_Updated.json') as f:
    ioc_dict = json.load(f)

#10 type of IOCs has been considered
file_hash = []
email_address =[]
ip_address=[]
url_address=[]
domain_address=[]
host_name=[]
region=[]
attack_technique=[]
attacker=[]
cve=[]

embedding_matrix=[]


for item in ioc_dict:
  for subitem in item['indicators']:
    if subitem['type']== 'FileHash-SHA1' or subitem['type']== 'FileHash-MD5' or subitem['type']== "FileHash-SHA256":
        file_hash.append(subitem['indicator'])
    elif subitem['type']== 'email':
        email_address.append(subitem['indicator'])
    elif subitem['type']== 'ipv4':
        ip_address.append(subitem['indicator'])
    elif subitem['type']== 'CVE':
        cve.append(subitem['indicator'])
    elif subitem['type']== 'url':
        url_address.append(subitem['indicator'])
    elif subitem['type']== 'domain':
        domain_address.append(subitem['indicator'])
    elif subitem['type']== 'hostname':
        host_name.append(subitem['indicator'])

  for subitem in range(len(item['targeted_countries'])):
    region.append(item['targeted_countries'][subitem])

  for subitem in range(len(item['attack_ids'])):
    attack_technique.append(item['attack_ids'][subitem])

  if item['adversary']=="" or item['adversary']=="Unknown":
    continue
  else:
    attacker.append(item['adversary'])


# tokenize IOCs
hash_tokens = [hs.split(' ') for hs in file_hash]
email_tokens = [email.split('@') for email in email_address]
ip_tokens = [ip.split('.') for ip in ip_address]
url_tokens = [dns.split('.') for dns in url_address]
domain_tokens = [dm.split('.') for dm in domain_address]
host_tokens = [dns.split('@') for dns in host_name]
region_tokens = [reg.split(' ') for reg in region]
attack_technique_tokens = [at.split('.') for at in attack_technique]
attacker_tokens = [atr.split('.') for atr in attacker]
cve_tokens = [cv.split('-') for cv in cve]

# train Word2Vec model

all_tokens = hash_tokens + email_tokens + ip_tokens + url_tokens + domain_tokens + host_tokens +region_tokens + attack_technique_tokens + attacker_tokens
model = Word2Vec(all_tokens, vector_size=len(all_tokens), window=5, min_count=1, workers=4)

#print(hash_tokens[0])


# generate embeddings
hash_embeddings = [model.wv[hs] for hs in hash_tokens]
embedding_matrix.append(hash_embeddings)
email_embeddings = [model.wv[em] for em in email_tokens]
embedding_matrix.append(email_embeddings)
ip_embeddings = [model.wv[ip] for ip in ip_tokens]
embedding_matrix.append(ip_embeddings)
url_embeddings = [model.wv[dns] for dns in url_tokens]
embedding_matrix.append(url_embeddings)
domain_embeddings = [model.wv[ip] for ip in domain_tokens]
embedding_matrix.append(domain_embeddings)
host_embeddings = [model.wv[ht] for ht in host_tokens]
embedding_matrix.append(host_embeddings)
cve_embeddings = [model.wv[chdtriv] for cv in cve_tokens]
embedding_matrix.append(cve_embeddings)
region_embeddings = [model.wv[reg] for reg in region_tokens]
embedding_matrix.append(region_embeddings)
attack_technique_embeddings = [model.wv[at] for at in attack_technique_tokens]
embedding_matrix.append(attack_technique_embeddings)
attacker_embeddings = [model.wv[atr] for atr in attacker_tokens]
embedding_matrix.append(attacker_embeddings)

all_embeddings1 = [model.wv[al] for al in all_tokens]

all_embeddings = []
for tokens in all_tokens:
    token_str = ' '.join(tokens)
    if token_str in model.wv:
        embedding = model.wv[token_str]
    else:
        # Use a placeholder vector for out-of-vocabulary tokens
        embedding = np.zeros(model.vector_size)
    all_embeddings.append(embedding.flatten())

all_embeddings = np.stack(all_embeddings)

#print(len(all_embeddings1))
print(len(all_embeddings))

268


In [2]:
#event generation for adjacency matrix
import json

with open('ICS_IOCs_Updated.json') as f:
    ioc_dict = json.load(f)

results = []

for item in ioc_dict:
  hash1 = []
  email1 =[]
  ip1= []
  url1= []
  domain1 =[]
  host1 =[]
  region1=[]
  attack_technique1=[]
  attacker1=[]
  cve1=[]

  res ={}
  for subitem in item['indicators']:
    if subitem['type'] == 'FileHash-SHA1' or subitem['type']== 'FileHash-MD5' or subitem['type'] == 'FileHash-SHA256':
        hash1.append(subitem['indicator'])
    elif subitem['type']== 'email':
        email1.append(subitem['indicator'])
    elif subitem['type']== 'CVE':
        cve1.append(subitem['indicator'])
    elif subitem['type']== 'ipv4':
        ip1.append(subitem['indicator'])
    elif subitem['type']== 'url':
        url1.append(subitem['indicator'])
    elif subitem['type']== 'domain':
        domain1.append(subitem['indicator'])
    elif subitem['type']== 'hostname':
        host1.append(subitem['indicator'])

  for subitem in range(len(item['targeted_countries'])):
    region1.append(item['targeted_countries'][subitem])

  for subitem in range(len(item['attack_ids'])):
    attack_technique1.append(item['attack_ids'][subitem])

  attacker1.append(item['adversary'])

  res['hash']=hash1
  res['email']=email1
  res['ip']=ip1
  res['url']=url1
  res['domain']=domain1
  res['host']=host1
  res['region']=region1
  res['attack_technique']=attack_technique1
  res['attacker']=attacker1
  res['Vulnerability']=cve1
  results.append(res)

with open('events.json', 'w') as ev:
    json.dump(results, ev, indent=4)


**Define Adjacency matrix**

In [3]:
import numpy as np

#print(hash1)
def com_adj_matrix(file_hash, email_address, ip_address, url_address, domain_address, host_name, region, attack_technique, attacker, cve
,events):

    nodes = file_hash + email_address + ip_address + url_address + domain_address + host_name + region + attack_technique + attacker + cve
    adj_matrix = np.zeros((len(nodes), len(nodes)), dtype=int)
    print("hash", len(file_hash))
    print("email", len(email_address))
    print("ip", len(ip_address))
    print("url", len(url_address))
    print("domain", len(domain_address))
    print("host", len(host_name))
    print("region", len(region))
    print("attack tech", len(attack_technique))
    print("attacker", len(attacker))
    print("cve", len(cve))
    print(nodes[0] in events[0]["hash"])

    # iterate over each pair of nodes and set the corresponding entry in the adjacency matrix
    for i in range(len(nodes)):
        for j in range(len(nodes)):
          for event in events:
            if nodes[i] in event["hash"] and nodes[j] in event["hash"]:
                adj_matrix[i,j] = 1
            if nodes[i] in event["email"] and nodes[j] in event["email"]:
                adj_matrix[i,j] = 1
            if nodes[i] in event["ip"] and nodes[j] in event["ip"]:
                adj_matrix[i,j] = 1
            if nodes[i] in event["url"] and nodes[j] in event["url"]:
                adj_matrix[i,j] = 1
            if nodes[i] in event["domain"] and nodes[j] in event["domain"]:
                adj_matrix[i,j] = 1
            if nodes[i] in event["host"] and nodes[j] in event["host"]:
                adj_matrix[i,j] = 1
            if nodes[i] in event["region"] and nodes[j] in event["region"]:
                adj_matrix[i,j] = 1
            if nodes[i] in event["attack_technique"] and nodes[j] in event["attack_technique"]:
                adj_matrix[i,j] = 1
            if nodes[i] in event["attacker"] and nodes[j] in event["attacker"]:
                adj_matrix[i,j] = 1
            if nodes[i] in event["Vulnerability"] and nodes[j] in event["Vulnerability"]:
                adj_matrix[i,j] = 1

                print(adj_matrix[i,j])
    return adj_matrix

**Compute Adjacency matrix**

In [4]:
mat=com_adj_matrix(file_hash, email_address, ip_address, url_address, domain_address, host_name, region, attack_technique, attacker, cve, results)
print(len(mat[0]))


hash 117
email 6
ip 1
url 1
domain 30
host 5
region 14
attack tech 89
attacker 5
cve 0
True
268


**Install tensorflow and tensorflow Hub**

In [ ]:
pip install tensorflow

In [ ]:
pip install tensorflow_hub

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the Universal Sentence Encoder model
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Function to create sentence embeddings using Universal Sentence Encoder
def create_embedding(sentence):
    # Convert sentence to a tensor
    sentence_tensor = tf.convert_to_tensor([sentence])

    # Generate embedding for the sentence
    embedding = use_model(sentence_tensor)

    return embedding.numpy()[0]  # Return the numpy array repre

**Install sentence transformer**

SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. The initial work is described in our paper Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks.

In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9162830020ad3521c4b0dbd70f0a2a9d4e667af872845f061645e116703d5120
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


**Generate relation**

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

def generate_relation(A_iocs, B_iocs, events):
    with open('ICS_IOCs_Updated.json') as f:
        data = json.load(f)

        event_A_iocs = []

        for event in events:
            for att_id in A_iocs:
                for item in data:
                    for indicator in item['indicators']:
                        if str(att_id) == str(indicator['indicator']):
                            if 'domain' in event and str(indicator['indicator']) in event['domain']:
                                event_A_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'hash' in event and str(indicator['indicator']) in event['hash']:
                                event_A_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'ip' in event and str(indicator['indicator']) in event['ip']:
                                event_A_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'email' in event and str(indicator['indicator']) in event['email']:
                                event_A_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'url' in event and str(indicator['indicator']) in event['url']:
                                event_A_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'host' in event and str(indicator['indicator']) in event['host']:
                                event_A_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'cve' in event and str(indicator['indicator']) in event['cve']:
                                event_A_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")


            if set(event['attacker']).intersection(A_iocs):
                event_A_iocs = []
                for item in data:
                    for i in range(len(A_iocs)):
                        if A_iocs[i]==item['adversary']:
                            event_A_iocs.append(item['id']+ ' ' +item['name']+' ' +item['description']+' ' +item['author_name']+' ' +item['created']+ ' ' + str(item['adversary'])+ ' ' +str(item['targeted_countries'])+ ' ' +str(item['tags'])+ ' ' +str(item['malware_families'])+ ' ' +str(item['attack_ids'])+ ' ' +str(item['industries']))


            if set(event['attack_technique']).intersection(A_iocs):
                event_A_iocs = []
                data_index = 0
                for att_id in A_iocs:
                    found = False
                    while data_index < len(data):
                        item = data[data_index]
                        if att_id in item['attack_ids']:
                            event_A_iocs.append(att_id+ ' ' +item['name']+' ' +item['description']+' ' +item['author_name']+' ' +item['created']+ ' ' + str(item['adversary'])+ ' ' +str(item['targeted_countries'])+ ' ' +str(item['tags'])+ ' ' +str(item['malware_families'])+ ' ' +str(item['industries']))
                            found = True
                            break
                        data_index += 1
                    data_index = 0 if found else data_index

            if set(event['region']).intersection(A_iocs):
                event_A_iocs = []
                data_index = 0
                for att_id in A_iocs:
                    found = False
                    while data_index < len(data):
                        item = data[data_index]
                        if att_id in item['targeted_countries']:
                            event_A_iocs.append(f"{att_id} {item['id']} {item['name']} {item['description']} {item['author_name']} {item['created']} {item['adversary']} {item['targeted_countries']} {item['tags']} {item['malware_families']} {item['industries']}")
                            found = True
                            break
                        data_index += 1
                    data_index = 0 if found else data_index


        event_B_iocs = []

        for event in events:
            for att_id in B_iocs:
                for item in data:
                    for indicator in item['indicators']:
                        if str(att_id) == str(indicator['indicator']):
                            if 'domain' in event and str(indicator['indicator']) in event['domain']:
                                event_B_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'hash' in event and str(indicator['indicator']) in event['hash']:
                                event_B_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'ip' in event and str(indicator['indicator']) in event['ip']:
                                event_B_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'email' in event and str(indicator['indicator']) in event['email']:
                                event_B_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'url' in event and str(indicator['indicator']) in event['url']:
                                event_B_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'host' in event and str(indicator['indicator']) in event['host']:
                                event_B_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")
                            if 'cve' in event and str(indicator['indicator']) in event['cve']:
                                event_B_iocs.append(f"{att_id} {indicator['id']} {indicator['type']} {indicator['created']} {indicator['content']} {indicator['title']} {indicator['description']}")


            if set(event['attacker']).intersection(B_iocs):
                event_B_iocs = []
                for item in data:
                    for i in range(len(B_iocs)):
                        if B_iocs[i]==item['adversary']:
                            event_B_iocs.append(item['id']+ ' ' +item['name']+' ' +item['description']+' ' +item['author_name']+' ' +item['created']+ ' ' + str(item['adversary'])+ ' ' +str(item['targeted_countries'])+ ' ' +str(item['tags'])+ ' ' +str(item['malware_families'])+ ' ' +str(item['attack_ids'])+ ' ' +str(item['industries']))


            if set(event['attack_technique']).intersection(B_iocs):
                event_B_iocs = []
                data_index = 0
                for att_id in B_iocs:
                    found = False
                    while data_index < len(data):
                        item = data[data_index]
                        if att_id in item['attack_ids']:
                            event_B_iocs.append(att_id+ ' ' +item['name']+' ' +item['description']+' ' +item['author_name']+' ' +item['created']+ ' ' + str(item['adversary'])+ ' ' +str(item['targeted_countries'])+ ' ' +str(item['tags'])+ ' ' +str(item['malware_families'])+ ' ' +str(item['industries']))
                            found = True
                            break
                        data_index += 1
                    data_index = 0 if found else data_index

            if set(event['region']).intersection(B_iocs):
                event_B_iocs = []
                data_index = 0
                for att_id in B_iocs:
                    found = False
                    while data_index < len(data):
                        item = data[data_index]
                        if att_id in item['targeted_countries']:
                            event_B_iocs.append(f"{att_id} {item['id']} {item['name']} {item['description']} {item['author_name']} {item['created']} {item['adversary']} {item['targeted_countries']} {item['tags']} {item['malware_families']} {item['industries']}")
                            found = True
                            break
                        data_index += 1
                    data_index = 0 if found else data_index

        #print(len(B_iocs))
        #print("event",len(event_B_iocs))

        #if set(event['attacker']).intersection(B_iocs):
        #    event_B_iocs = []
        #    for item in data:
        #        for i in range(len(B_iocs)):
        #            if B_iocs[i]==item['adversary']:
        #                event_B_iocs.append(item['id']+ ' ' +item['name']+' ' +item['description']+' ' +item['author_name']+' ' +item['created']+ ' ' + str(item['adversary'])+ ' ' +str(item['targeted_countries'])+ ' ' +str(item['tags'])+ ' ' +str(item['malware_families'])+ ' ' +str(item['attack_ids'])+ ' ' +str(item['industries']))

        #print("event", event_A_iocs)
        # Generate embeddings
        model = SentenceTransformer('distilbert-base-nli-mean-tokens')
        A_embeddings = model.encode(event_A_iocs)
        B_embeddings = model.encode(event_B_iocs)
        embeddingA = np.array(A_embeddings)
        embeddingB = np.array(B_embeddings)

        print(embeddingA.shape)
        print(embeddingB.shape)

        num_features = max(embeddingA.shape[1], embeddingB.shape[1])
        embeddingA = np.pad(embeddingA, ((0, 0), (0, num_features - embeddingA.shape[1])), mode='constant')
        embeddingB = np.pad(embeddingB, ((0, 0), (0, num_features - embeddingB.shape[1])), mode='constant')
        similarity_matrix = cosine_similarity(embeddingA, embeddingB)

    return similarity_matrix


**Relation definition**

In [ ]:
rel=[]

#Relation (Adjacency matriX): R1: attacker uses hash
R1 = generate_relation(attacker, file_hash, results)
rel.append(R1)

#Relation (Adjacency matriX): R2: attacker accesses email
R2 = generate_relation(attacker, email_address, results)
rel.append(R2)

#Relation (Adjacency matriX): R3: attacker accesses Ip adress
R3 = generate_relation(attacker, ip_address, results)
rel.append(R3)

#Relation (Adjacency matriX): R4: attacker accesses domain adress
R4 = generate_relation(attacker, domain_address, results)
rel.append(R4)

#Relation (Adjacency matriX): R5: attacker accseses url
R5 = generate_relation(attacker, url_address, results)
rel.append(R5)

#Relation (Adjacency matriX): R6: attacker accesses host adress
R6 = generate_relation(attacker, host_name, results)
rel.append(R6)

#Relation (Adjacency matriX): R7: attacker assists another attacker
R7 = generate_relation(attacker, attacker, results)
rel.append(R7)

#Relation (Adjacency matriX): R8: attacker terget region
R8 = generate_relation(attacker, region, results)
rel.append(R8)

#Relation (Adjacency matriX): R9: attacker uses technique
R9 = generate_relation(attacker, attack_technique, results)
rel.append(R9)

#Relation (Adjacency matriX): R10: ip connects to email_address
R10 = generate_relation(email_address, ip_address, results)
rel.append(R10)

#Relation (Adjacency matriX): R11: domain_name resolve to ip
R11 = generate_relation(domain_address, ip_address, results)
rel.append(R11)

#Relation (Adjacency matriX): R12: domain_adrdess registered by email_address
R12 = generate_relation(domain_address, email_address, results)
rel.append(R12)

#Relation (Adjacency matriX): R13: vulverability evolve vulnerability
#R13 = generate_relation(cve, cve, results)
#rel.append(R13)

#print(R4)


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

(5, 768)
(237, 768)
(5, 768)
(6, 768)
(5, 768)
(1, 768)
(5, 768)
(90, 768)
(5, 768)
(1, 768)
(5, 768)
(5, 768)
(5, 768)
(5, 768)
(5, 768)
(14, 768)
(5, 768)
(89, 768)
(6, 768)
(1, 768)
(90, 768)
(1, 768)
(90, 768)
(6, 768)


**Meta-Path and Meta-Graph**

In [ ]:
M=[]
#M1: Attacker-hash-attacker
M1 = np.dot(R1, R1.transpose())
M.append(M1)

#M2: attacker-email-email
M2 = np.dot(R2, R2.transpose())
M.append(M2)

#M3: attacker-ip-email
M3 = np.dot(R3, R3.transpose())
M.append(M3)

#M4: attacker-domain-email
M4 = np.dot(R4, R4.transpose())
M.append(M4)

#M5: attacker-url-email
M5 = np.dot(R5, R5.transpose())
M.append(M5)

#M6: attacker-host-email
M6 = np.dot(R6, R6.transpose())
M.append(M6)

#M7: attacker-attacker-attacker
M7 = np.dot(R7, R7.transpose())
#M.append(M7)

#M8: attacker-region-attacker
M8 = np.dot(R8, R8.transpose())
#M.append(M8)

#M9: attacker-technique-attacker
M9 = np.dot(R9, R9.transpose())
#M.append(M9)

#M10: email-ip-email
M10 = np.dot(R10, R10.transpose())
#M.append(M10)

#M11: domain-ip-domain
M11 = np.dot(R11, R11.transpose())
#M.append(M11)

#M12: domain-email-domain
M12 = np.dot(R12, R12.transpose())
#M.append(M12)

#M13: vul-vul-vul
#M13 = np.dot(R13, R13.transpose())
#M.append(M13)


#M16: A-E-IP-E-A
M16 = np.dot(np.dot(R2,np.dot(R10, R10.transpose())),R2.transpose())
#M.append(M16)

print("M", M)
#print(len(adjacency_matrix))

M [array([[39.686657, 45.526134, 37.382328, 38.196228, 57.16509 ],
       [45.526134, 52.371376, 42.947872, 43.90358 , 65.75915 ],
       [37.382328, 42.947872, 35.29425 , 36.011208, 53.911457],
       [38.196228, 43.90358 , 36.011208, 36.841293, 55.10434 ],
       [57.16509 , 65.75915 , 53.911457, 55.10434 , 82.67555 ]],
      dtype=float32), array([[1.9091188, 1.898715 , 1.7339128, 1.6826633, 2.0991657],
       [1.898715 , 1.8906282, 1.7256093, 1.6746547, 2.0886388],
       [1.7339128, 1.7256093, 1.5755787, 1.5291585, 1.9069254],
       [1.6826633, 1.6746547, 1.5291585, 1.484677 , 1.8513489],
       [2.0991657, 2.0886388, 1.9069254, 1.8513489, 2.3102832]],
      dtype=float32), array([[0.12974317, 0.15480362, 0.11879618, 0.14324947, 0.20060262],
       [0.15480362, 0.1847046 , 0.14174217, 0.17091872, 0.23934987],
       [0.11879618, 0.14174217, 0.10877284, 0.1311629 , 0.18367691],
       [0.14324947, 0.17091872, 0.1311629 , 0.15816177, 0.22148539],
       [0.20060262, 0.23934987, 0.1

**compute number of meta-paths and meta-graphs**

In [ ]:
import numpy as np

def num_meta_paths(commuting_matrix, adj_matrix_A, adj_matrix_B, start_node, end_node):
    adj_matrix_A = np.array(adj_matrix_A)
    adj_matrix_B = np.array(adj_matrix_B)
    commuting_matrix = np.array(commuting_matrix)
    num_paths = 0
    for i in range(int(adj_matrix_A.shape[0]-250)):

        for j in range(int(adj_matrix_B.shape[0]-250)):
            #print("hi1")
            for k in range(commuting_matrix.shape[1]):
                #print("hi-k")
                if adj_matrix_A[i,start_node] and adj_matrix_B[j,end_node] and commuting_matrix[i%commuting_matrix.shape[1],j%commuting_matrix.shape[1]] and adj_matrix_B[j,k] and adj_matrix_A[i,k]:
                    num_paths += commuting_matrix[i%commuting_matrix.shape[1],j%commuting_matrix.shape[1]] * adj_matrix_A[i,k] * adj_matrix_B[j,k]
                    #print("hi-if", num_paths)
        #print("hi2")
    #print(num_paths)
    return num_paths

In [ ]:
import numpy as np

def num_meta_paths(commuting_matrix, adj_matrix_A, adj_matrix_B, start_node, end_node):
    adj_matrix_A = np.array(adj_matrix_A)
    adj_matrix_B = np.array(adj_matrix_B)
    commuting_matrix = np.array(commuting_matrix)
    num_paths = 0

    indices_A = np.where(adj_matrix_A[:, start_node] > 0)[0]
    indices_B = np.where(adj_matrix_B[:, end_node] > 0)[0]
    commuting_mod = np.arange(commuting_matrix.shape[0]) % commuting_matrix.shape[1]
    for i in indices_A:
        for j in indices_B:
            num_paths += np.sum(
                commuting_matrix[commuting_mod[i%commuting_matrix.shape[1]], commuting_mod[j%commuting_matrix.shape[1]]] *
                adj_matrix_A[i, :] *
                adj_matrix_B[j, :]
            )
    #print("num_path", num_paths)

    return num_paths

In [ ]:
import tensorflow as tf

def num_meta_paths(commuting_matrix, adj_matrix_A, adj_matrix_B, start_node, end_node):
    adj_matrix_A = tf.convert_to_tensor(adj_matrix_A, dtype=tf.float32)
    adj_matrix_B = tf.convert_to_tensor(adj_matrix_B, dtype=tf.float32)
    commuting_matrix = tf.convert_to_tensor(commuting_matrix, dtype=tf.float32)
    num_paths = 0



    indices_A = tf.where(adj_matrix_A[:, start_node] > 0)[:, 0]
    indices_B = tf.where(adj_matrix_B[:, end_node] > 0)[:, 0]
    commuting_mod = tf.range(commuting_matrix.shape[0]) % commuting_matrix.shape[1]

    for i in indices_A:
        for j in indices_B:
            num_paths += tf.reduce_sum(
                commuting_matrix[commuting_mod[i % commuting_matrix.shape[1]], commuting_mod[j % commuting_matrix.shape[1]]] *
                adj_matrix_A[i, :] *
                adj_matrix_B[j, :]
            )

    return num_paths




**Genrate Meta-paths and meta-graph lists**

In [ ]:
import numpy as np

def meta_path_list(M,mat):
    meta_path1 = []
    for i in range(len(mat)):
        print("----------------------i", i)
        row_results = []
        for j in range(len(mat)):
            num_paths = num_meta_paths(M, mat, mat, i, j)
            row_results.append(num_paths)
        meta_path1.append(row_results)
    print("meta-path", meta_path1)
    return meta_path1


In [ ]:
print(len(M))
print(len(all_embeddings))
path_list=meta_path_list(M[1],all_embeddings)
print("last",path_list)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(path_list[0])

[[1090.0, 1090.0, 1090.0, 1090.0, 1090.0, 1090.0, 1090.0, 1090.0, 1090.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1090.0, 1090.0, 1090.0, 1090.0, 1090.0, 1090.0, 1090.0, 1090.0, 1090.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Define MPGIIS

In [ ]:

def compute_miis(path_list):
    m = [[0 for col in range(len(path_list[0]))] for row in range(len(path_list[0]))]
    for i in range(len(path_list[0])):
      for j in range(len(path_list[0][0])):
        for k in range(len(path_list)):
            miis= 0
            num_p_ii = path_list[k][i][i]
            num_p_jj = path_list[k][j][j]
            num_p_ij =path_list[k][i][j]
            if num_p_ii == 0 or num_p_jj == 0:
                continue
            miis += (2 * num_p_ij) / (num_p_ii + num_p_jj)
        m[i][j]=miis
    #print("m", m)
    return m

**Compute MPGIIS**

In [ ]:
#Compute MIIS
print(len(path_list))
MIIS= compute_miis(path_list)
print("miis", MIIS)

13
m [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

#Feature matrix generation

In [ ]:

import json
import pandas as pd
from datetime import datetime

with open('ICS_IOCs_Updated.json') as f:
    threats = json.load(f)

organizations = list(set([org for threat in threats for org in threat['industries']]))
locations = list(set([loc for threat in threats for loc in threat['targeted_countries']]))
feature_matrix = pd.DataFrame(columns= ['active_time', 'update_frequency']+ organizations + locations )

for threat in threats:
    threat_id = threat['name']
    active_time = (datetime.now() - datetime.strptime(threat['created'], "%Y-%m-%dT%H:%M:%S.%f")).days
    update_frequency = threat['revision']
    for indicator in threat['indicators']:
      if indicator['type']=='FileHash-SHA1'or indicator['type']=='FileHash-MD5' or  indicator['type'] == 'FileHash-SHA256' or  indicator['type'] == 'email' or  indicator['type'] == 'CVE' or  indicator['type'] == 'ipv4'or  indicator['type'] == 'url'or  indicator['type'] == 'domain'or  indicator['type'] == 'hostname' :
        row = {'active_time': active_time, 'update_frequency': update_frequency}
        for org in organizations:
            row[org] = 1 if org in threat['industries'] else 0
        for loc in locations:
            row[loc] = 1 if loc in threat['targeted_countries'] else 0
        feature_matrix = pd.concat([feature_matrix, pd.DataFrame([row])], ignore_index=True)

    for tech in threat['attack_ids']:
      row = {'active_time': active_time, 'update_frequency': update_frequency}
      for org in organizations:
            row[org] = 1 if org in threat['industries'] else 0
      for loc in locations:
            row[loc] = 1 if loc in threat['targeted_countries'] else 0
      feature_matrix = pd.concat([feature_matrix, pd.DataFrame([row])], ignore_index=True)

    for reg in threat['targeted_countries']:
      row = { 'active_time': active_time, 'update_frequency': update_frequency}
      for org in organizations:
            row[org] = 1 if org in threat['industries'] else 0
      for loc in locations:
            row[loc] = 1 if loc in threat['targeted_countries'] else 0
      feature_matrix = pd.concat([feature_matrix, pd.DataFrame([row])], ignore_index=True)
    if threat['adversary']=="" or threat['adversary']=="Unknown":
      continue
    else:
        row = { 'active_time': active_time, 'update_frequency': update_frequency}
        for org in organizations:
            row[org] = 1 if org in threat['industries'] else 0
        for loc in locations:
            row[loc] = 1 if loc in threat['targeted_countries'] else 0
    feature_matrix = pd.concat([feature_matrix, pd.DataFrame([row])], ignore_index=True)

feature_matrix = feature_matrix[['active_time', 'update_frequency'] + organizations + locations ]
print(feature_matrix[:18])


    active_time update_frequency Energy Bank Shipping Logistics  \
0            51                1      0    0        0         1   
1            51                1      0    0        0         1   
2            51                1      0    0        0         1   
3            51                1      0    0        0         1   
4            51                1      0    0        0         1   
..          ...              ...    ...  ...      ...       ...   
261         121                1      0    0        0         0   
262         121                1      0    0        0         0   
263         121                1      0    0        0         0   
264         121                1      0    0        0         0   
265         121                1      0    0        0         0   

    Transportation Healthcare Healthcare and Social Assistance Services  \
0                1          0                                         1   
1                1          0                

# Compute GCN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

#adjacency_matrix = np.random.rand(266, 266)  # Example random adjacency matrix
#feature_matrix = np.random.rand(266, 32)  # Example random feature matrix

adjacency_matrix = MIIS

adj = torch.tensor(adjacency_matrix, dtype=torch.float32)
feature_matrix = feature_matrix.apply(pd.to_numeric, errors='coerce')

# Convert feature_matrix to a PyTorch tensor.
feature_tensor = torch.tensor(feature_matrix.values, dtype=torch.float32)

class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.W = nn.Linear(in_features, out_features)

    def forward(self, input: torch.Tensor, adj: torch.Tensor) -> torch.Tensor:
        support = torch.mm(adj, input)  # Perform matrix multiplication between adjacency matrix and input features
        output = self.W(support)  # Apply linear transformation
        return output

# Create an instance of the GraphConvolution class.
gcn = GraphConvolution(32, 32)

# Pass the input data to the forward method.
output = gcn(feature_tensor, adj)
print(output)

tensor([[-0.0707, -0.0789, -0.0574,  ...,  0.1050, -0.0869,  0.1627],
        [-0.0707, -0.0789, -0.0574,  ...,  0.1050, -0.0869,  0.1627],
        [-0.0707, -0.0789, -0.0574,  ...,  0.1050, -0.0869,  0.1627],
        ...,
        [-0.0707, -0.0789, -0.0574,  ...,  0.1050, -0.0869,  0.1627],
        [-0.0707, -0.0789, -0.0574,  ...,  0.1050, -0.0869,  0.1627],
        [-0.0707, -0.0789, -0.0574,  ...,  0.1050, -0.0869,  0.1627]],
       grad_fn=<AddmmBackward0>)


In [ ]:
import torch

# Assuming 'output' is the tensor containing the output values

# Calculate the mean along the specified dimension
aggregated_output = torch.mean(output, dim=1)

# Print the aggregated output
print(aggregated_output)

tensor([-0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056, -0.0056,
        -0.0056, -0.0056, -0.0056, -0.00

In [ ]:

import networkx as nx
import torch
import numpy as np

# Example GCN output
gcn_output = output

# Convert tensor to a regular NumPy array

gcn_output = gcn_output.detach().numpy()

# Create a graph from the GCN output
G = nx.Graph()
num_nodes = len(gcn_output)
G.add_nodes_from(range(1, num_nodes + 1))

# Add edges based on GCN output
for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        if np.greater(gcn_output[i], gcn_output[j]):
            G.add_edge(i + 1, j + 1)

# Calculate the degree centrality of nodes
degree_centrality = nx.degree_centrality(G)

# Print the degree centrality of each node
for node, centrality in degree_centrality.items():
    print(f"Node {node}: Degree Centrality = {centrality}")

NameError: ignored